# some parameters

In [1]:
! python --version

Python 2.7.6


In [2]:
import warnings
import os.path

from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
from __future__ import absolute_import, division, print_function, unicode_literals

import os, csv

import matplotlib.pyplot as plt

    
warnings.filterwarnings("ignore", category=DeprecationWarning)
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
cate_list = [7052]
for cate in cate_list:
    if cate in [11922]:
        pred_date = pd.to_datetime('2017-10-10')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [7052, 7054]:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
    else:
        pred_date = pd.to_datetime('2018-03-10')
        scenario['lookforwardPeriodDays'] = 8
scenario['forecastStartDate'] = pred_date

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
pd.options.display.max_columns = 1000

In [4]:
p1_raw_columns=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']

In [5]:
to_del = ['areatypearray', 'productdesc', 'mainproductkey', 'bd_subtype1_giveaway', 'bd_subtype4_threshold1',
           'bd_subtype4_giveaway1', 'bd_subtype4_threshold2', 'bd_subtype4_giveaway2', 'bd_subtype4_threshold3',
           'bd_subtype4_giveaway3', 'bd_subtype2_threshold', 'bd_subtype2_giveaway', 'bd_subtype2_maximumgiveaway',
           'bd_subtype15_thresholdnumber1', 'bd_subtype15_giveawayrate1', 'bd_subtype15_thresholdnumber2', 'bd_subtype15_giveawayrate2',
           'bd_subtype15_thresholdnumber3', 'bd_subtype15_giveawayrate3', 'bd_subtype6_thresholdnumber', 'bd_subtype6_freenumber', 'fg_productkey',
           'dd_percent', 'bd_subtype1_percent', 'bd_subtype4_percent', 'bd_subtype2_percent', 'bd_subtype15_percent',
           'bd_subtype6_percent', 'suit_percent', 'fg_percent', 'allpercentdiscount']

In [6]:
p2_df_columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
             'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
             'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
             'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
             'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

In [7]:
ts_df_columns=['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']

# Functions

In [8]:
def dealwithnull(period_promo_raw):
    period_promo_raw.replace('null', np.nan, inplace=True)
    period_promo_raw.replace(-999, np.nan, inplace=True)
    period_promo_raw.replace(' null', np.nan, inplace=True)
    period_promo_raw.replace('None', np.nan, inplace=True)

In [9]:
def f01(period_promo_raw):
    period_promo_raw.replace('null', np.nan, inplace=True)
    period_promo_raw.replace(-999, np.nan, inplace=True)
    period_promo_raw.replace(' null', np.nan, inplace=True)
    period_promo_raw.replace('None', np.nan, inplace=True)
    # to_float_col = ['productkey', 'promo_inperiod_flag', 'mainproductkey', 'hierarchylevel1key', 'hierarchylevel2key', 'hierarchylevel3key', 'statuscode', 'promotionkey', 'jdprice', 'syntheticgrossprice', 'promotiontype', 'promotionsubtype', 'levelmember', 'directdiscount_productkey', 'directdiscount_maxjdprice', 'directdiscount_minjdprice', 'directdiscount_avgjdprice', 'directdiscount_maxscrapedprice', 'directdiscount_minscrapedprice', 'directdiscount_avgscrapedprice', 'directdiscount_discount', 'directdiscount_price', 'directdiscount_limitationflag', 'directdiscount_limitationcode', 'directdiscount_availabilitynumberflag', 'directdiscount_availabilitynumber', 'directdiscount_minimumnumberflag', 'directdiscount_minimumnumber', 'directdiscount_maximumnumberflag', 'directdiscount_maximumnumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway', 'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3', 'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1', 'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3', 'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool', 'suit_discount', 'freegift_productkey', 'freegift_maxscrapedprice', 'freegift_minscrapedprice', 'freegift_avgscrapedprice', 'freegift_maxjdprice', 'freegift_minjdprice', 'freegift_avgjdprice', 'freegift_freegiftvalue', 'freegift_freegiftvalueatjdprice', 'freegift_freegiftvalueatscrapedprice', 'directdiscount_saleprice', 'directdiscount_percent', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent', 'bundle_subtype6_percent', 'suit_percent', 'freegift_percent', 'allpercentdiscount']
    to_float_col = ['allpercentdiscount','bundle_subtype15_giveawayrate1','bundle_subtype15_giveawayrate2','bundle_subtype15_giveawayrate3','bundle_subtype15_percent','bundle_subtype15_thresholdnumber1','bundle_subtype15_thresholdnumber2','bundle_subtype15_thresholdnumber3','bundle_subtype1_giveaway','bundle_subtype1_percent','bundle_subtype1_threshold','bundle_subtype2_giveaway','bundle_subtype2_maximumgiveaway','bundle_subtype2_percent','bundle_subtype2_threshold','bundle_subtype4_giveaway1','bundle_subtype4_giveaway2','bundle_subtype4_giveaway3','bundle_subtype4_percent','bundle_subtype4_threshold1','bundle_subtype4_threshold2','bundle_subtype4_threshold3','bundle_subtype6_freenumber','bundle_subtype6_percent','bundle_subtype6_thresholdnumber','directdiscount_availabilitynumber','directdiscount_discount','directdiscount_saleprice','hierarchylevel3key','jdprice','mainproductkey','productkey','promotionkey','promotionsubtype','promotiontype','statuscode','suit_avgvaluepool','suit_discount','suit_maxvaluepool','suit_minvaluepool','suit_percent','syntheticgrossprice']
    period_promo_raw['enddatetime'] = pd.to_datetime(period_promo_raw['enddatetime'])
    period_promo_raw['startdatetime'] = pd.to_datetime(period_promo_raw['startdatetime'])

    if 'object' in period_promo_raw[to_float_col].dtypes.values:
        object2Float(period_promo_raw,to_float_col)
    return period_promo_raw

In [10]:
def checknullproportion(df):
    count = len(df) - df.count()
    return count/len(df)

In [11]:
def f01_p2(promoCalendarDf, scenario, PredictEndDate=None, ForecastStartDate=None, train=True):
    if ForecastStartDate == None:
        ForecastStartDate = pd.to_datetime(scenario['forecastStartDate'])
    
    DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])
    promoCalendarDf['Date'] = pd.to_datetime(promoCalendarDf['Date'])
    promoCalendarDf['dt'] = pd.to_datetime(promoCalendarDf['dt'])
    dealwithnull(promoCalendarDf)
    promoCalendarDf.drop_duplicates(inplace=True)
    
    # Convert Object -> float
    if 'object' in promoCalendarDf.dtypes.values:
        obj_cols = get_column_by_type(promoCalendarDf,'object')
        object2Float(promoCalendarDf,obj_cols)
    if train:
        promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']<ForecastStartDate)&(promoCalendarDf['Date']>DataStartDate)]
    else:
        if PredictEndDate == None:
            PredictEndDate = ForecastStartDate + datatime.timedelta(days=scenario['lookforwardPeriodDays'])
        promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']>=ForecastStartDate)&(promoCalendarDf['Date']<=PredictEndDate)]
    columns_list = promoCalendarDf.columns.tolist()
    if columns_list[0] == 'ProductKey' and columns_list[1] == 'Date':
        neworder = [columns_list[1]] + [columns_list[0]] + list(columns_list[2:])
        promoCalendarDf = promoCalendarDf[neworder]
    return promoCalendarDf

## Feature selection random forest (beta)

In [24]:
def clearbycheckvariance(df):
    """
    clear zero variance columns
    The first columns is Date, Date type has no variance, so just keep it in df!
    """
    enum_df = df.select_dtypes(exclude=['int64','float64'])
    num_df = df.select_dtypes(include=['int64', 'float64'])
#     print(enum_df.columns.values, len(enum_df.columns.values))
#     print(num_df.columns.values, len(num_df.columns.values))
    var = num_df.var(axis=0)
    zero_var_features = var.index[var==0]
#     print(zero_var_features.values)
    to_keep = num_df.columns.difference(zero_var_features).values
#     print(to_keep, len(to_keep))
    print ('number of zero variance columns: {}'.format(len(zero_var_features)))
#     print (df.shape)
    num_df = num_df[to_keep]
#     num_df = num_df.loc[:, var != 0]
#     print('No.1',new_df.columns,len(new_df.columns))
#     print('No.2',num_df.columns,len(num_df.columns))
#     if set(new_df.columns.values) == set(num_df.columns.values):
#         print('#-__-Fine!')
#     else:
#         print('#- _ - Bad news!')
#         return False
    df = pd.concat([enum_df, num_df],axis=1)
    print(zero_var_features.values)
    print ('shape of new dataframe: ',df.shape)
    return df, to_keep, zero_var_features.values

In [13]:
# -*- coding: utf-8 -*-
def feature_select(data, Ycol='sales', k=10, project_path='tmp/data/'):
    from sklearn.ensemble import RandomForestClassifier
    
    col_row = data.shape

    row = col_row[0]
    col = col_row[1]

    X= data.drop(Ycol,axis=1)
    
    Y= data[Ycol]
    Y.values.reshape(len(Y),-1)

    Z = pd.concat([X,Y], axis = 1)
    y=Y.ravel()

    forest = RandomForestClassifier()
    forest.fit(X.values, y)

    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    cut_indices = indices[0:k]

    header_indices = np.copy(cut_indices)
    topk_indices = np.append(header_indices,[-1])


    out = data.iloc[:,topk_indices]
    out.to_csv(project_path + "RF_top{}.csv".format(k),index=False)

    # Print the feature ranking
    print("Feature ranking:")
    for count in range(k):
        print("feature name:{}".format(Z.columns[header_indices[k]]))
        print("%d. no.%d (%f)" % (count+1, indices[count], importances[indices[count]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(k), importances[cut_indices],color="b", yerr=std[cut_indices], align="center")
    plt.xticks(range(k), cut_indices)
    plt.xlim([-1, k])
    plt.show()
    return out

In [17]:
ts_df

NameError: name 'ts_df' is not defined

# data read in and preprocessing

## p1-f01 preprocessing

In [27]:
path = 'tmp/data/'
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']
features_to_keep = []
# cate_list = [7054, 7052, 870]
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_f01
for fw in for_what:
    for cate in cate_list:
        print ("output and save: %s_p1_%s" %(str(cate),fw))
        p1_path = os.path.join(path, str(cate)+'_'+item+suffix)
        
        p1_used_header=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
        print('read data from '+p1_path+'....')
        period_promo_raw = pd.read_csv(p1_path,sep='\t',header=None)
        period_promo_raw.columns=p1_used_header
        period_promo_raw_clean = period_promo_raw
        train_pred_gate = fw   # 'train'
        if cate in [11922]:
            pred_date = pd.to_datetime('2017-10-10')
            scenario['lookforwardPeriodDays'] = 7
        if cate in [7052, 7054]:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        else:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        f01_output = ow_f01.generate_f01_promo(area_rdc_map, period_promo_raw_clean,scenario, train_pred_gate, ForecastStartDate=pred_date)
#         f01_output.set_index('Date', inplace=True)
        if fw == 'train':
            print('#---------clear period by check variance:......--------#####')
            f01_output, features_to_keep,features_to_del = clearbycheckvariance(f01_output)
            print('features to keep:', features_to_keep)
            print('features to delete:',features_to_del)
            scenario['selectedColumns']['features'] = list(set(scenario['selectedColumns']['features']) - set(features_to_del))
            print('#---------clear by check variance complete------#####')
        else:
            f01_output = f01_output[features_to_keep]
        print('saving data to '+os.path.join(path, train_pred_gate+'_'+item+'_'+str(cate)+'_2018_02_01ws.csv'))
        f01_output.to_csv(os.path.join(path, train_pred_gate+'_'+item+'_'+str(cate)+'_2018_02_01ws.csv'),index=False)
        print('-------saving complete---------')

output and save: 7052_p1_train
read data from tmp/data/7052_p1.da....
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Floa

## ts-df readin and preprocessing

In [ ]:
# ## ts_df preprocessing ####
# ForecastStartDate = pd.to_datetime(scenario['forecastStartDate'])
# DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])
# ts_df['Date'] = pd.to_datetime(ts_df['Date'])
# ts_df_pred = ts_df[(ts_df['Date']<ForecastStartDate)&(ts_df['Date']>DataStartDate)]
# ts_df_pred.replace('null', np.nan, inplace=True)
# ts_df_pred.replace(-999, np.nan, inplace=True)
# ts_to_float_col = ['RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
# if 'object' in ts_df_pred[ts_to_float_col].dtypes.values:
#     object2Float(ts_df_pred,ts_to_float_col)

# Train

In [22]:
promoCalendarDf.set_index("Date")

,dt,ProductKey,PromotionCount,bundlecount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,SyntheticGrossPrice,promotionkey,promotiontype,promotionsubtype,syntheticgrossprice_vb,jdprice,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,bundleflag,directdiscountflag,numberproducts,strongmark,stockprice
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-12,2017-04-12,4139150,2,1,0.398693,0.089037,0.243865,-0.0514,-0.5929,-0.322150,0.089037,0.089037,0.089037,0.398693,0.398693,0.398693,1050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1248.59
2017-04-12,2017-04-12,4264350,2,0,0.031792,0.002890,0.017341,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.031792,0.002890,0.017341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,341.55
2017-04-12,2017-04-12,692602,1,0,0.058824,0.058824,0.058824,-0.0407,-0.0407,-0.040700,0.000000,0.000000,0.000000,0.058824,0.058824,0.058824,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,111.61
2017-04-12,2017-04-12,1337994,1,0,0.338369,0.338369,0.338369,-0.1061,-0.1061,-0.106100,0.000000,0.000000,0.000000,0.338369,0.338369,0.338369,198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,282.00
2017-04-12,2017-04-12,1482370,2,0,0.088235,0.064706,0.076471,0.0882,0.0647,0.076450,0.000000,0.000000,0.000000,0.088235,0.064706,0.076471,170.0,197978988.0,1.0,16.0,170.0,170.0,0.0882,14.0,1.0,0.0,1.0,1,0,159.15
2017-04-12,2017-04-12,497498,2,0,0.250000,0.232143,0.241071,0.1711,0.1513,0.161200,0.000000,0.000000,0.000000,0.250000,0.232143,0.241071,152.0,197592877.0,1.0,101.0,152.0,168.0,0.1711,24.0,12.0,0.0,1.0,1,0,151.19
2017-04-12,2017-04-12,497505,4,0,0.360215,0.250000,0.277554,0.2013,0.0638,0.098175,0.000000,0.000000,0.000000,0.360215,0.250000,0.277554,149.0,197967667.0,1.0,101.0,149.0,186.0,0.2013,7.0,2.0,0.0,1.0,1,0,165.54
2017-04-12,2017-04-12,2165620,3,0,0.257732,0.231959,0.249141,0.0336,0.0000,0.022400,0.000000,0.000000,0.000000,0.257732,0.231959,0.249141,298.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,322.86
2017-04-12,2017-04-12,2563234,1,0,0.141129,0.141129,0.141129,0.1411,0.1411,0.141100,0.000000,0.000000,0.000000,0.141129,0.141129,0.141129,248.0,197434188.0,1.0,101.0,248.0,248.0,0.1411,24.0,12.0,0.0,1.0,1,0,193.80


In [18]:
###train###
path = 'tmp/data/'
result_path = 'tmp/data/wusai/'
# cate_list = [870]
#cate_list = [7052, 7054, 870]
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
'''online analysis'''
#scenario['use_cart_feature'] = False
#scenario['use_promo_features'] = False
p2_to_keep = []
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_train
for cate in cate_list:
    if cate in [11922]:
        pred_date = pd.to_datetime('2017-10-10')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [7052, 7054]:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
    else:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
        #pred_date = pd.to_datetime('2018-03-10')
        #scenario['lookforwardPeriodDays'] = 8

    pro_canlender_path = os.path.join(path, str(cate)+'_'+'p2.da')
    promoCalendarDf = pd.read_csv(pro_canlender_path,sep='\t',header=None)
    promoCalendarDf.columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
             'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
             'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
             'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
             'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

    promoCalendarDf = f01_p2(promoCalendarDf,scenario)
#     promoCalendarDf.set_index('Date', inplace = True)
#     print(promoCalendarDf.info())
    print('###--------clear promoCalendarDf by check variance.......-----######')
    promoCalendarDf, p2_to_keep, p2_to_del = clearbycheckvariance(promoCalendarDf)
    features_to_keep = np.append(features_to_keep, p2_to_keep)
    print('features to delete:', p2_to_del)
    print('features_to_keep:', features_to_keep)
    scenario['selectedColumns']['features'] = list(set(scenario['selectedColumns']['features']) - set(p2_to_del))
    print('####--------clear complete------------###########')
    
    p1_out_path = os.path.join(path, 'train'+'_p1_'+str(cate)+'_2018_02_01ws.csv')
    period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

#     sea_path = os.path.join(path, str(cate)+'_season.csv')
    sea_path = os.path.join(path, '870_season.csv')
    seasonality_df = pd.read_csv(sea_path, parse_dates=['Date'])

    file_path = os.path.join(path, str(cate)+'_'+'ts.da')
    ts_df = pd.read_csv(file_path,header=None,sep='\t')
    model,feature=ow_train.train(area_rdc_map,period_promo_raw,promoCalendarDf,ts_df,scenario,holidays_df,seasonality_df,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
    #feature.to_csv(os.path.join(result_path, str(cate)+'_train_feature_with_season_180201_0515.csv'),index=False)
    print('save model to'+os.path.join(result_path, str(cate)+'_train_model_with_season_180201_0515'+'.pkl'))
    save_object(model, os.path.join(result_path, str(cate)+'_train_model_with_season_180201_0515'+'.pkl'))

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag
convert to Float columns:
stockprice
<class 'pandas.core.frame.DataFrame'>
Int64Index: 190247 entries, 691 to 456773
Data columns (total 40 columns):
Date                      190247 non-null datetime64[ns]
ProductKey                190247 non-null int64
HierarchyLevel3Key        190247 non-null int64
Promotion

ow_train.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace('null', np.nan, inplace=True)
ow_train.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace(-999, np.nan, inplace=True)


convert to Float columns:
RDCKey
convert to Float columns:
ProductKey


code/refactor/common.py:2178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype('float')


convert to Float columns:
HierarchyLevel1Key
convert to Float columns:
HierarchyLevel2Key
convert to Float columns:
HierarchyLevel3Key
convert to Float columns:
brand_code
convert to Float columns:
sales
convert to Float columns:
priceAfterDiscount
convert to Float columns:
jd_prc
convert to Float columns:
vendibility
convert to Float columns:
counterState
convert to Float columns:
salesForecast
convert to Float columns:
reserveState
convert to Float columns:
stockQuantity
convert to Float columns:
utc_flag
use cart feature?
False
| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - A

| - Rolling decay 3 mean...
| - Added rolling features in 4.168189 seconds 

| - Added rolling features in 159.287591 seconds 

| Calculating volume rolling features...
('| - Rolling value:', 'salesForecast')
| - Shift 365...
| - Shift 180...
| - Shift 90...
| - Shift 30...
| - Shift 7...
| - Added rolling features in 15.868805 seconds 

featuresDf shape
(2231828, 130)
rdc
3.0


TypeError: float() argument must be a string or a number

# predict

In [69]:
# -*- coding: utf-8 -*-

scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)

#scenario['lookforwardPeriodDays'] = 1
#scenario['use_cart_feature'] = False
#scenario['use_promo_features'] = False
path = 'tmp/data/'
result_path = 'tmp/data/wusai/'
# cate_list = [7054]#[7054]
#cate_list = [7052, 7054, 870]

holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_predict
#cate = 7052
for cate in cate_list:
    pro_canlender_path = os.path.join(path, str(cate)+'_'+'p2.da')
    promoCalendarDf = pd.read_csv(pro_canlender_path,sep='\t',header=None)
    promoCalendarDf.columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
           'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
           'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
           'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
           'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

    #p1_out_path = os.path.join(path, 'predict'+'_p1_'+str(cate)+'_2018_02_01.csv')
    p1_out_path = os.path.join(path, 'predict'+'_p1_'+str(cate)+'.csv')
    period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

    file_path = file_path = os.path.join(path, str(cate)+'_'+'ts.da')
    ts_df = pd.read_csv(file_path,header=None,sep='\t')

    model_path = os.path.join(result_path, str(cate)+'_train_model'+'_with_season_180201_0515'+'.pkl')
    with open(model_path,'r') as input:
        model = pickle.load(input)

    sea_path = os.path.join(path, '870_season.csv')
    seasonality_df = pd.read_csv(sea_path, parse_dates=['Date'])

    if cate in [11922]:
        pred_date = pd.to_datetime('2017-10-10')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [7052, 7054]:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
    else:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
        #pred_date = pd.to_datetime('2018-03-10')
        #scenario['lookforwardPeriodDays'] = 30
    result,df_fut=ow_predict.predict(area_rdc_map,period_promo_raw,promoCalendarDf,ts_df,scenario,holidays_df,model,seasonality_df,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
    #result.to_csv('tmp/data/0425_result/'+'result_'+str(cate)+'.csv',index=False)
    result.to_csv(result_path+'result_'+str(cate)+'_zero_promo_0515.csv',index=False)

IOError: [Errno 2] No such file or directory: u'tmp/data/wusai/7052_train_model_with_season_180201_0515.pkl'

# analysis

In [ ]:
###anlysis###
import pandas as pd
import numpy as np
df1 = pd.read_csv(result_path+'result_'+str(cate)+'_zero_promo_0515.csv') #target ?? where is target？
df1['Date'] = pd.to_datetime(df1['Date'])
# df2 = pd.read_csv('tmp/11922_2018/result_no_promo_11922_171201_180322.csv') #target
cols = ['Date','HierarchyLevel3Key','ProductKey','RDCKey','ypred']
df11 = df1[cols]
# print df11
# df22 = df2[cols]

# file_path = '/home/ubuntu/yulong/promotion_offline/tmp/11922_2018/11922_predict_ts'
# ts1_df = pd.read_csv(file_path,header=None,sep='\t')
# ts1_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
#ts1_df['Date'] = pd.to_datetime(ts1_df['Date'])
groupkeys = ['Date','HierarchyLevel3Key','ProductKey','RDCKey']

df11 = pd.merge(df11[groupkeys+['ypred']],ts_df[groupkeys+['salesForecast']],how='left',on=groupkeys)
# df22 = pd.merge(df22[groupkeys+['ypred']],ts_df[groupkeys+['salesForecast']],how='left',on=groupkeys)

df111 = df11.groupby(['HierarchyLevel3Key','ProductKey','RDCKey']).sum().reset_index()
# df222 = df22.groupby(['HierarchyLevel3Key','ProductKey','RDCKey']).sum().reset_index()
df111.RDCKey.drop_duplicates()
print "###promo###"
for rdc in list(df111.RDCKey.unique()):
#for rdc in [3,4,5,6,9,10,316,772]:
    this_df = df111[df111.RDCKey==rdc]
    this_df['resi'] = np.abs(this_df['salesForecast']-this_df['ypred'])
    print "rdc:" + str(rdc)
    print this_df['resi'].sum()/this_df['salesForecast'].sum()
# print "###no_promo###"
# for rdc in list(df222.RDCKey.unique()):
# #for rdc in [3,4,5,6,9,10,316,772]:
#     this_df = df222[df222.RDCKey==rdc]
#     this_df['resi'] = np.abs(this_df['salesForecast']-this_df['ypred'])
#     print "rdc:" + str(rdc)
#     print this_df['resi'].sum()/this_df['salesForecast'].sum()